In [16]:
import os
from pyannote.audio import Pipeline
import whisper
import torch

In [1]:

# ============= Настройки =============
file_name = "32140-312ds213-91094-2134.mp3"   # путь к аудио
num_speakers = 4                             # количество спикеров
model_name = "base"                          # whisper: tiny / base / small / medium / large

# Hugging Face токен
hf_auth_token = ""

# ============= Загрузка моделей =============
print("Загрузка моделей...")
model = whisper.load_model(model_name)


Загрузка моделей...


NameError: name 'whisper' is not defined

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=hf_auth_token
)

pipeline = pipeline.to(device)

# ============= Диаризация =============
print("Запуск диаризации...")
diarization = pipeline(file_name, num_speakers=num_speakers)

# Собираем сегменты по спикерам
speaker_segments = []
for turn, _, speaker in diarization.itertracks(yield_label=True):
    speaker_segments.append({
        'speaker': speaker,
        'start': turn.start,
        'end': turn.end
    })



Запуск диаризации...


/home/debral/DevSpace/small_transcriptor/.venv/lib/python3.13/site-packages/pyannote/audio/utils/reproducibility.py:74: ReproducibilityWarning: TensorFloat-32 (TF32) has been disabled as it might lead to reproducibility issues and lower accuracy.
It can be re-enabled by calling
   >>> import torch
   >>> torch.backends.cuda.matmul.allow_tf32 = True
   >>> torch.backends.cudnn.allow_tf32 = True
See https://github.com/pyannote/pyannote-audio/issues/1370 for more details.

  warnings.warn(
/home/debral/DevSpace/small_transcriptor/.venv/lib/python3.13/site-packages/pyannote/audio/models/blocks/pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /pytorch/aten/src/ATen/native/ReduceOps.cpp:1839.)
  std = sequences.std(dim=-1, correction=1)
/home/debral/DevSpace/small_transcriptor/.venv/lib/python3.13/site-packages/torchaudio/_backend/soundfile_backend.py:71

In [ ]:

# ============= Транскрибация =============
print("Запуск транскрибации...")
result = model.transcribe(
    file_name,
    language="ru",
    verbose=True,
    word_timestamps=True
)

# ============= Сопоставление спикеров =============
print("Сопоставление спикеров...")
for segment in result['segments']:
    segment_start = segment['start']
    segment_end = segment['end']

    possible_speakers = []
    for speaker_seg in speaker_segments:
        overlap_start = max(speaker_seg['start'], segment_start)
        overlap_end = min(speaker_seg['end'], segment_end)
        if overlap_start < overlap_end:
            overlap_duration = overlap_end - overlap_start
            possible_speakers.append({
                'speaker': speaker_seg['speaker'],
                'overlap_duration': overlap_duration
            })

    speaker_label = "Спикер неизвестен"
    if possible_speakers:
        main_speaker = max(possible_speakers, key=lambda x: x['overlap_duration'])
        speaker_label = main_speaker['speaker']
        if len(possible_speakers) > 1:
            speaker_label += " (основной)"

    print(f"[{speaker_label}] [{segment_start:.1f}-{segment_end:.1f}с]: {segment['text']}")